### Config

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from huggingface_hub import hf_hub_download
import torch
from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.model.moirai import MoiraiForecast
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

/Users/tomaltenborg/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/tomaltenborg/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2BD1B165-EC09-3F68-BCE4-8FE4E70CA7E2> /Users/tomaltenborg/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F8622D92-25A9-3A61-A089-C917FDA36C1B> /Users/tomaltenborg/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [41]:
SIZE = "base"  # model size: choose from {'small', 'base', 'large'}
PDT = 6  # prediction length: any positive integer
CTX = 200  # context length: any positive integer
PSZ = "auto"  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 32  # batch size: any positive integer
TEST = 10  # test set length: any positive integer

In [42]:
import pandas as pd
# Read data into pandas DataFrame
year = pd.read_parquet('/Users/tomaltenborg/Documents/Master/Master thesis/Notebooks/M3 Data/M3_quarter_processed.parquet')
year = year[['Series', 'Date', 'Value']]

# Assuming you have a DataFrame named 'df' with a column named 'date'
year['Date'] = pd.to_datetime(year['Date'])  # Convert the 'date' column to datetime if it's not already

# Set the 'date' column to quarter end
year['Date'] = year['Date'] + pd.offsets.QuarterEnd()

year.set_index('Date', inplace=True)
# year_try = year.loc[(year['Series'] == 1)]
# year_try.index.year

# Convert into GluonTS dataset
ds = PandasDataset.from_long_dataframe(year, item_id='Series', target = 'Value')


In [43]:

# Prepare pre-trained model by downloading model weights from huggingface hub
model = MoiraiForecast.load_from_checkpoint(
    checkpoint_path=hf_hub_download(
        repo_id=f"Salesforce/moirai-1.0-R-{SIZE}", filename="model.ckpt"
    ),
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=10,
    target_dim=1,
    feat_dynamic_real_dim=ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=ds.num_past_feat_dynamic_real,
    map_location="cuda:0" if torch.cuda.is_available() else "cpu",
)

predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(ds)

forecast_it = iter(forecasts)
forecast = next(forecast_it)
# forecast = next(forecast_it)


model.ckpt:   0%|          | 0.00/365M [00:00<?, ?B/s]

In [45]:
forecast

gluonts.model.forecast.SampleForecast(info=None, item_id='646', samples=array([[6943.9365, 6238.2305, 6624.611 , 6021.879 , 5671.247 , 5887.771 ],
       [5709.958 , 6226.2725, 6505.9126, 6269.2065, 6779.601 , 6703.12  ],
       [6003.205 , 5813.9805, 6199.698 , 6177.29  , 5670.4893, 5618.0137],
       [6058.6177, 5852.019 , 5637.1685, 6204.155 , 5900.543 , 7014.543 ],
       [5962.0312, 5791.894 , 5975.5947, 6043.4453, 6051.0283, 5968.6353],
       [6451.703 , 4489.9194, 6281.37  , 5879.628 , 6628.004 , 6880.6045],
       [5988.9443, 6219.71  , 5839.0205, 6214.1367, 5576.917 , 3995.3696],
       [5933.4463, 5741.0137, 5969.624 , 6342.9893, 5772.3257, 5886.439 ],
       [5894.872 , 6264.8115, 5972.7266, 6121.327 , 5862.8545, 6273.8013],
       [6012.566 , 6538.1597, 5570.7983, 6020.7285, 5444.902 , 7461.9253]],
      dtype=float32), start_date=Period('1995Q2', 'Q-OCT'))

In [40]:
try:
    prediction = next(forecast_it)
    print(prediction)
except StopIteration:
    print("No more predictions.")


gluonts.model.forecast.SampleForecast(info=None, item_id='678', samples=array([[5930.1064, 5765.3486, 6068.126 , 5925.6885, 6012.6606, 6085.9146],
       [5953.232 , 6099.0957, 6004.3154, 5933.5005, 5901.115 , 5802.386 ],
       [5941.8926, 5790.126 , 5979.6245, 5862.4985, 6155.8843, 5829.305 ],
       [6022.2964, 5972.741 , 5978.5605, 5794.41  , 5970.3984, 5870.0337],
       [5981.071 , 5933.5137, 6135.1426, 5930.8135, 5961.005 , 5948.469 ],
       [5839.4766, 6060.993 , 5932.593 , 5949.047 , 5949.5967, 5960.837 ],
       [6028.8477, 5643.6704, 5996.956 , 5908.5024, 5654.1157, 6006.108 ],
       [6013.6255, 5964.8726, 6216.2783, 6182.545 , 5899.8345, 6044.2695],
       [5936.557 , 5948.8633, 5914.338 , 5974.927 , 6111.9653, 5918.5957],
       [5832.415 , 5926.651 , 6024.466 , 5973.6147, 5943.2783, 5969.2754]],
      dtype=float32), start_date=Period('1995Q2', 'Q-OCT'))


In [46]:
import pandas as pd

# Assume 'forecast' is your SampleForecast object from GluonTS
samples = forecast.samples
start_date = pd.Period(forecast.start_date, freq=forecast.start_date.freq)
forecast_horizon = samples.shape[1]

# Create date range for the forecast
date_range = pd.period_range(start=start_date, periods=forecast_horizon, freq=start_date.freq)

# Convert samples to DataFrame
forecast_df = pd.DataFrame(samples.T, index=date_range)

# Example of how to handle multiple forecasts
# Assuming 'forecasts' is a list of SampleForecast objects
all_forecasts = []
for forecast in forecasts:
    samples = forecast.samples
    start_date = pd.Period(forecast.start_date, freq=forecast.start_date.freq)
    forecast_horizon = samples.shape[1]
    date_range = pd.period_range(start=start_date, periods=forecast_horizon, freq=start_date.freq)
    forecast_df = pd.DataFrame(samples.T, index=date_range)
    all_forecasts.append(forecast_df)

# Concatenate all forecast DataFrames if multiple series forecasts are handled
complete_forecast_df = pd.concat(all_forecasts, axis=1)


In [47]:
complete_forecast_df #not at all what I want, will fix later

0            1            2            3            4  \
1995Q2  4038.486328  4428.572266  4334.152832  4370.912598  4297.304199   
1995Q3  4298.679199  4137.354980  4414.893555  4350.182617  4380.300781   
1995Q4  4363.674316  4167.795898  4636.836914  4188.086426  4441.843262   
1996Q1  4599.052734  4683.175781  4297.367188  4484.309082  4874.330078   
1996Q2  4913.558594  4280.154297  3890.277344  4200.827148  4619.156250   
...             ...          ...          ...          ...          ...   
1962Q1          NaN          NaN          NaN          NaN          NaN   
1962Q2          NaN          NaN          NaN          NaN          NaN   
1962Q3          NaN          NaN          NaN          NaN          NaN   
1970Q4          NaN          NaN          NaN          NaN          NaN   
1971Q1          NaN          NaN          NaN          NaN          NaN   

                  5            6            7            8            9  \
1995Q2  4287.392578  4326.892578  4394.171875  4310.711426  4440.833984   
1995Q3  4390.496582  4371.315430  4350.246582  4264.393555  4662.462402   
1995Q4  4486.935547  4129.214355  4262.779785  4389.775391  4413.237305   
1996Q1  4422.441895  4784.047363  4632.289062  4334.060547  4311.744141   
1996Q2  4144.167480  4418.671387  4404.056641  5746.585938  4763.798828   
...             ...          ...          ...          ...          ...   
1962Q1          NaN          NaN          NaN          NaN          NaN   
1962Q2          NaN          NaN          NaN          NaN          NaN   
1962Q3          NaN          NaN          NaN          NaN          NaN   
1970Q4          NaN          NaN          NaN          NaN          NaN   
1971Q1          NaN          NaN          NaN          NaN          NaN   

                  0            1            2            3            4  \
1995Q2  6354.082520  5922.824219  6059.158203  5848.829102  5678.653320   
1995Q3  5883.614746  5630.755859  5037.222168  6050.657715  5932.266602   
1995Q4  5689.836914  5312.616699  6193.322266  6223.175293  5575.269531   
1996Q1  5715.770508  5951.703125  5757.941406  5259.254883  5784.081543   
1996Q2  5665.853516  6469.897461  4819.901855  6031.134766  6435.949707   
...             ...          ...          ...          ...          ...   
1962Q1          NaN          NaN          NaN          NaN          NaN   
1962Q2          NaN          NaN          NaN          NaN          NaN   
1962Q3          NaN          NaN          NaN          NaN          NaN   
1970Q4          NaN          NaN          NaN          NaN          NaN   
1971Q1          NaN          NaN          NaN          NaN          NaN   

                  5            6             7            8            9  \
1995Q2  5780.344727  5711.405273   5632.809570  5641.053711  5695.183594   
1995Q3  5995.532227  5567.112793   5958.007324  5713.115234  6099.678711   
1995Q4  5107.619629  6087.265137   5762.980469  5984.690430  6076.606445   
1996Q1  4614.487305  5668.734863   8259.591797  6086.259766  5545.311523   
1996Q2  6915.045410  5887.838867  11300.269531  6766.326172  4037.102539   
...             ...          ...           ...          ...          ...   
1962Q1          NaN          NaN           NaN          NaN          NaN   
1962Q2          NaN          NaN           NaN          NaN          NaN   
1962Q3          NaN          NaN           NaN          NaN          NaN   
1970Q4          NaN          NaN           NaN          NaN          NaN   
1971Q1          NaN          NaN           NaN          NaN          NaN   

                  0            1            2            3            4  \
1995Q2  2747.969482  3204.644531  3107.585938  2630.009033  3175.960938   
1995Q3  2982.084717  2628.688965  3055.290771  3463.535889  3953.345947   
1995Q4  2498.909180  2276.610352  2896.695312  2768.575195  3128.352539   
1996Q1  2555.961914  2615.659668  3151.967773  2887.109863  1798.899414   
1996Q2  3192.539307  2099.2